**1.** Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?

1.1 Предпочтительно использовать precision@k, т. к. она проста в интерпретации и нагляднее демонстрирует эффективность работы рекомендательной системы.

1.2 Для рассмотренной на уроке метрики hit_rate@k можно рассмотреть следующие варианты использования:

а) Магазин дорогих/элитных/единичных товаров высокой стоимости. Тут достаточно чтобы клиент (покупатель) купил какой-либо товар в магазине.

б) Возможно, имеет смысл применять на каком-либо видеосервисе. Так как после просмотра основного видео клиент (покупатель) важно чтобы остался на сервисе и продолжил смотреть другие видео (товар), неважно какие.

***

**2.** В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.

2.1 В ранжировании разница позиций в начале списка намного существеннее разницы позиций в конце списка (например, между 1 и 2 позициями разница более существенна чем между 9 и 10) и логарифм как раз обеспечивает данное требование.

2.2 Экспоненту

2.3 Кросс-энтропия и максимальная апостериорная (MAP) оценка

***

**3.** Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

**MAE** - Среднее абсолютное отклонение (Интеракция - оценка по непрерывной шкале. Например, рейтинг товара);

**MSE** - Среднеквадратичная ошибка (Интеракция - оценка по непрерывной шкале. Например, рейтинг товара);

**F1-Measure** - Среднее гармоническое метрик Precision и Recall. Полезно, когда заранее невозможно сказать, какая из метрик важнее (Интеракция - бинарные данные. Например, купил товар или нет, поставил положительный или отрицательный отзыв);

**ROC AUC** - Насколько высока концентрация интересных товаров в начале списка рекомендаций (Интеракция - бинарные данные. Например, купил товар или нет, поставил положительный или отрицательный отзыв);

**Spearman Correlation** - Корреляция (Спирмена) реального и прогнозируемого рангов рекомендаций (Интеракция - любая. Оценивает правильность порядка рекомендаций);

**Fraction of Concordance Pairs** - Насколько высока концентрация интересных товаров в начале списка рекомендаций (Интеракция - любая. Оценивает правильность порядка рекомендаций);

***

**4.** boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8 (expected reciprocal rank)  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

In [1]:
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

$pr@8 = \frac{0 + 1 + 1 + 0 + 1 + 1 + 0 + 0}{8} = 50\%$

In [2]:
import numpy as np

In [3]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
        
    precision = flags.sum() / len(recommended_list)    
    
    return precision

In [4]:
precision_at_k(recommended, boughted, 8)

0.5

*******

$rec@8 = \frac{0 + 1 + 1 + 0 + 1 + 1 + 0 + 0}{6} = 66\%$

In [5]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [6]:
recall_at_k(recommended, boughted, 8)

0.6666666666666666

***

$AP@8=\frac{0+\frac{1}{2}+\frac{2}{3}+0+\frac{3}{5}+\frac{4}{6}+0+0}{8}=0.304$

In [7]:
def ap_k(recommended_list, bought_list, k=5):
    
    if len(recommended_list) > k:
        recommended_list = recommended_list[:k]
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(k): 
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / len(flags)
    
    return result

In [8]:
ap_k(recommended, boughted, 8)

0.30416666666666664

***

$RR@8=\frac{0}{1}+\frac{1}{2}=0.5$

In [9]:
def reciprocal_rank_at_k(recommended_list, bought_list, k=5):
    if len(recommended_list) > k:
        recommended_list = recommended_list[:k]
    ranks = 0.
    for item_rec in recommended_list:
        if item_rec not in bought_list:
            ranks += 1
            continue
        else:
            ranks += 1
            return 1 / ranks

In [10]:
reciprocal_rank_at_k(recommended, boughted, 8)

0.5

***

$DCG@8 = \frac{1}{8}*(0 / 1 + 1 / log(2) + 1 / log(3) + 0 / log(4) + 1 / log(5) + 1 / log(6) + 0 / log(7) + 0 / log(8))=0.306$

$idealDCG@8 = \frac{1}{8}*(1 / 1 + 1 / log(2) + 1 / log(3) + 1 / log(4) + 1 / log(5) + 1 / log(6) + 1 / log(7) + 1 / log(8))=0.58$

$NDCG@8 = \frac{0.306}{0.58}=0.527$

In [11]:
def dcg_at_k(recommended_list, bought_list, k=5):
    if len(recommended_list) > k:
        recommended_list = recommended_list[:k+1]
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    sum_ = flags[0]/1
    
    for i in range(2, k+1):
        value = flags[i-1]/np.log2(i)
        sum_ += value
        
    return sum_/k

In [12]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    if len(recommended_list) > k:
        recommended_list = recommended_list[:k+1]
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    sum_ = 1
    
    for i in range(2, k+1):
        value = 1/np.log2(i)
        sum_ += value
    ideal_dcg = sum_ / k
    result = dcg_at_k(recommended_list, bought_list, k) / ideal_dcg
    
    return result

In [13]:
ndcg_at_k(recommended, boughted, 8)

0.5279127445637272

***

$ERR@8 = 0+1*\frac{\frac{2^1-1}{2}}{2}+0.5*\frac{\frac{2^1-1}{2}}{3}+0+0.25*\frac{\frac{2^1-1}{2}}{5}+0.125*\frac{\frac{2^1-1}{2}}{6}+0+0+0+0.0625*\frac{\frac{2^1-1}{2}}{8}=0.37$

In [14]:
def err_at_k(recommended_list, bought_list, k=5):
    if len(recommended_list) > k:
        recommended_list = recommended_list[:k]
    flags = np.isin(recommended_list, bought_list).astype(float)
    p = 1
    err = 0
    for i in range(1, k+1):
        r = (2**flags[i-1] - 1) / 2**np.max(flags)
        err += p * r / i
        p *= 1 - r
    return err

In [15]:
err_at_k(recommended, boughted, 8)

0.36875